In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from __future__ import division
import math
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np 
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.transforms as T
from model import Model_defs
from collections import defaultdict
from coco_dict import COCO_Dict


import matplotlib.pyplot as plt
import matplotlib.patches as patches

import torch.utils.data as data
from PIL import Image
import os.path


to_img= T.Compose([T.ToPILImage()])
to_tensor = T.Compose([T.ToTensor()])
#load_norm = T.Compose([T.Resize((hps.image_size,hps.image_size)),T.ToTensor(),T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
device = torch.device('gpu' if torch.cuda.is_available() else 'cpu')

'''
TODO:

1. Add Pixelshuffle
2. Add Self.attention
3. Test the model on GPU

'''

In [ ]:
# os.makedirs('./data/coco')
# os.makedirs('./data/coco/anno')
dict = {1:'apple',96:'water'}

In [ ]:
class Parser():
    #hyperparameters
    def __init__(self):
        self.img_size = 416
        self.img_path = './data/coco/val2017' # folder
        self.ann_path = './data/coco/annotations/instances_val2017.json' # file name
        self.img_channels = 3
        self.ignore_thres = 0.5
        self.lambda_coord = 1
        self.batch_size = 5
        self.num_anchors = 3
        self.num_classes = 80
        self.max_objs = 50
        self.model_name = 'YOLOv3'
        self.model_path = './'+ self.model_name +'/Model/'
        self.img_save_path = './'+self.model_name+'/Image/'
        self.lr = 0.02
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.n_epoch = 30
        self.loss_show_freq = 200
        self.img_show_freq = 200
        self.model_save_freq = 1000
        
args = Parser()

class Upsample(nn.Module):
    def __init__(self, size = None, scale_factor = 2, mode ='nearest', align_corners = None):
        super().__init__()
        self.size = size
        self.scale_factor = scale_factor
        self.mode = mode
        self.align_corners = align_corners
    def forward(self, x):
        return F.interpolate(x, self.size, self.scale_factor, self.mode, self.align_corners)


if not os.path.exists(args.model_path):
    os.makedirs(args.model_path)
if not os.path.exists(args.img_save_path):
    os.makedirs(args.img_save_path)



In [ ]:
class COCODataset(Dataset):

    def __init__(self, root, annFile, transform=None, target_transform=None):
        from pycocotools.coco import COCO
        self.root = root
        self.coco = COCO(annFile)
        self.ids = list(self.coco.imgs.keys())
        self.transform = transform
        self.target_transform = target_transform
        self.resize = T.Compose([T.ToPILImage(),T.Resize((args.img_size,args.img_size)),T.ToTensor()])
        self.max_objs = args.max_objs
        
        # Map from 1~90 to 0~79
        self.ninety_to_eighty = list(np.arange(0,80))
        for i in [0,12,26,29,30,45,66,68,69,71,83]:
            self.ninety_to_eighty.insert(i,None)
        
    def __getitem__(self, index):
        
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anno = coco.loadAnns(ann_ids)

        path = coco.loadImgs(img_id)[0]['file_name']

        img = Image.open(os.path.join(self.root, path)).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(anno)
        
        _, h, w = img.shape
        dim_diff = abs(h-w)
        pad1, pad2 = dim_diff // 2, dim_diff - dim_diff // 2
        pad = (0,0,pad1, pad2,0,0) if h <= w else (pad1, pad2,0,0,0,0)
        img = F.pad(img, pad, mode='constant', value=0.5)
        _, padded_h, padded_w = img.shape
        img = self.resize(img)
        
        bbox = torch.tensor([i['bbox'] for i in anno]) # x,y,w,h
        cat = [self.ninety_to_eighty[i['category_id']] for i in anno]
        if (len(cat) == 0):
            return self.__getitem__(torch.randint(len(self),(1,1)).int().item())
        category_id = torch.LongTensor(cat).float().unsqueeze(1)

        labels = torch.cat([category_id, bbox], dim = 1)
            
            
        labels[:,1] = labels[:,1] + pad[0]
        labels[:,2] = labels[:,2] + pad[3]
        
        labels[:,1:] *= (args.img_size/ padded_h)
        
        labels[:,1] += labels[:,3]/2
        labels[:,2] += labels[:,4]/2
        
        filled_labels = torch.zeros((self.max_objs, 5))
        if labels is not None:
            filled_labels[range(len(labels))[:self.max_objs]] = labels[:self.max_objs]
        
        # the output will become scaled Class, CenterX, CenterY, Gw, Gh
        return  path , img, filled_labels

    def __len__(self):
        return len(self.ids)

In [ ]:
class ImageDataset(Dataset):

    def __init__(self, root, transform = None):
        from glob import glob
        self.root = root 
        self.files = sorted(glob(os.path.join(root, '*.jpg')))
        self.transform = transform
        self.resize = T.Compose([T.ToPILImage(),T.Resize((args.img_size,args.img_size)),T.ToTensor()])

    def __getitem__(self, index):

        path = self.files[index % len(self.files)]
        img = Image.open(path)
        if self.transform is not None:
            img = self.transform(img)
        
        _, h, w = img.shape
        dim_diff = abs(h-w)
        pad1, pad2 = dim_diff // 2, dim_diff - dim_diff // 2
        pad = (0,0,pad1, pad2,0,0) if h <= w else (pad1, pad2,0,0,0,0)
        img = F.pad(img, pad, mode='constant', value=0.5)
        _, padded_h, padded_w = img.shape
        img = self.resize(img)
        path = os.path.relpath(path, start= self.root)
        return  path, img

    def __len__(self):
        return len(self.files)

In [ ]:
def bbox_iou(box1, box2, x1y1x2y2=False):
    """
    Returns the IoU of two bounding boxes
    """
    if not x1y1x2y2:
        # Transform from center and width to exact coordinates
        b1_x1, b1_x2 = box1[:, 0] - box1[:, 2] / 2, box1[:, 0] + box1[:, 2] / 2
        b1_y1, b1_y2 = box1[:, 1] - box1[:, 3] / 2, box1[:, 1] + box1[:, 3] / 2
        b2_x1, b2_x2 = box2[:, 0] - box2[:, 2] / 2, box2[:, 0] + box2[:, 2] / 2
        b2_y1, b2_y2 = box2[:, 1] - box2[:, 3] / 2, box2[:, 1] + box2[:, 3] / 2
    else:
        # Get the coordinates of bounding boxes
        b1_x1, b1_y1, b1_x2, b1_y2 = box1[:,0], box1[:,1], box1[:,2], box1[:,3]
        b2_x1, b2_y1, b2_x2, b2_y2 = box2[:,0], box2[:,1], box2[:,2], box2[:,3]

    # get the corrdinates of the intersection rectangle
    inter_rect_x1 =  torch.max(b1_x1, b2_x1)
    inter_rect_y1 =  torch.max(b1_y1, b2_y1)
    inter_rect_x2 =  torch.min(b1_x2, b2_x2)
    inter_rect_y2 =  torch.min(b1_y2, b2_y2)
    # Intersection area
    inter_area =    torch.clamp(inter_rect_x2 - inter_rect_x1 + 1, min=0) * \
                    torch.clamp(inter_rect_y2 - inter_rect_y1 + 1, min=0)
    # Union Area
    b1_area = (b1_x2 - b1_x1 + 1) * (b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1) * (b2_y2 - b2_y1 + 1)

    iou = inter_area / (b1_area + b2_area - inter_area + 1e-16)

    return iou


def non_max_suppression(prediction, num_classes, conf_thres=0.5, nms_thres=0.4):
    """
    Removes detections with lower object confidence score than 'conf_thres' and performs
    Non-Maximum Suppression to further filter detections.
    Returns detections with shape:
        (x, y, w, h, object_conf, class_score, class_pred)
    """
    
    output = [None for _ in range(len(prediction))] # len = args.batch_size
    for image_i, image_pred in enumerate(prediction):
        # Filter out confidence scores below threshold
        conf_mask = (image_pred[:, 4] >= conf_thres).squeeze()
        image_pred = image_pred[conf_mask]
        # If none are remaining => process next image
        if not image_pred.size(0):
            continue
        # Get score and class with highest confidence
        class_conf, class_pred = torch.max(image_pred[:, 5:5 + num_classes], 1,  keepdim=True)
        # Detections ordered as (x1, y1, x2, y2, obj_conf, class_conf, class_pred)
        detections = torch.cat((image_pred[:, :5], class_conf.float(), class_pred.float()), 1)
        # Iterate through all predicted classes
        unique_labels = detections[:, -1].cpu().unique()
        if prediction.is_cuda:
            unique_labels = unique_labels.cuda()
        for c in unique_labels: # certain class
            # Get the detections with the particular class
            detections_class = detections[detections[:, -1] == c]
            # Sort the detections by maximum objectness confidence
            
            
            _, conf_sort_index = torch.sort(detections_class[:, 4], descending=True)
            detections_class = detections_class[conf_sort_index]
            # Perform non-maximum suppression
            max_detections = []
            while detections_class.size(0):
                # Get detection with highest confidence and save as max detection
                max_detections.append(detections_class[0].unsqueeze(0))
                # Stop if we're at the last detection
                if len(detections_class) == 1:
                    break
                # Get the IOUs for all boxes with lower confidence
                ious = bbox_iou(max_detections[-1], detections_class[1:])
                # Remove detections with IoU >= NMS threshold
                detections_class = detections_class[1:][ious < nms_thres]

            max_detections = torch.cat(max_detections).data
            # add max detection to the certain place of the list
            output[image_i] = max_detections if output[image_i] is None else torch.cat((output[image_i], max_detections))

    return output

def plot_on_o_img(path, input_labels, input_classes, save = False, step ='Train'):
    '''
    input labels should be x,y,w,h
    '''
    labels = input_labels.clone()
    classes = input_classes.clone()
    cmap = plt.get_cmap('tab20b')
    colors = [cmap(i) for i in np.linspace(0, 1, args.num_classes)]
    
    for p, label, cls in zip(path, labels, classes):
        s_img_p = os.path.join(args.img_path, p)
        img = to_tensor(Image.open(s_img_p))
        
        plt.figure()
        fig, ax = plt.subplots(figsize= (8,8))
        ax.imshow(to_img(img))
        
        pad_x = max(img.shape[1] - img.shape[2], 0)//2
        pad_y = max(img.shape[2] - img.shape[1], 0)//2
        
        idx = torch.any(label[...,2:4].byte(),-1)
        label = label[idx]
        
        if not label.size(0):
            print('None Object Detected')
            plt.show()
            continue
            
        label[:,0] -= label[:,2]/2
        label[:,1] -= label[:,3]/2
        label[:,:] *= (max(img.shape[1:])/args.img_size)
        label[:,0] -= pad_x
        label[:,1] -= pad_y
             
        

        for lab, c in zip(label, cls):
            x1,y1,w,h = lab
            bbox = patches.Rectangle((x1, y1), w, h, linewidth=2,
                                edgecolor= colors[int(c)],facecolor='none')
            
            ax.add_patch(bbox)
            plt.text(x1, y1, s=str(COCO_Dict.id_to_cat[int(c)]), color='white', verticalalignment='top',
                            bbox={'color': colors[int(c)], 'pad': 0})
        plt.show()
            
        if save:
            fig.savefig(args.img_save_path + args.model_name + p[:-4] +"_pred_"+str(step)+".png")

In [ ]:
def build_targets(pred_boxes, target, anchors, num_anchors, num_classes, dim, ignore_thres, img_dim):
    '''
    Based on 416x416 coordinate.
    '''
    nB = target.size(0)
    nA = num_anchors
    nC = num_classes
    dim = dim
    n_grid = dim 
    grid_size = args.img_size / n_grid 
    # The feature map is (B, num_anchors, g_dim, g_dim, self.bbox_attrs) Pred_boxes

    mask        = torch.zeros(nB, nA, dim, dim)
    conf_mask   = torch.ones(nB, nA, dim, dim)
    tx          = torch.zeros(nB, nA, dim, dim)
    ty          = torch.zeros(nB, nA, dim, dim)
    tw          = torch.zeros(nB, nA, dim, dim)
    th          = torch.zeros(nB, nA, dim, dim)
    tconf       = torch.zeros(nB, nA, dim, dim)
    tcls        = torch.zeros(nB, nA, dim, dim, num_classes)

    nGT = 0 # number of ground truth
    nCorrect = 0
    # 將target轉換為Feature map
    for b in range(nB): # every batch
        for t in range(target.shape[1]): # every bbox from target 
            if target[b, t].sum() == 0:
                continue
            nGT += 1
            # Convert to position relative to box
            gx = target[b, t, 1]
            gy = target[b, t, 2]
            gw = target[b, t, 3]
            gh = target[b, t, 4]
            # Get grid box indices
            gi = (gx/ grid_size).long()
            gj = (gy/ grid_size).long()

            # Get shape of gt box
            gt_box = torch.tensor([0,0,gw,gh]).unsqueeze(0)
            # Get shape of anchor box
            # input is scaled anchors
            anchor_shapes = torch.cat([torch.zeros((len(anchors),2)),torch.tensor(anchors).float()], dim =1 )
            # Calculate iou between gt and anchor shapes
            anch_ious = bbox_iou(gt_box, anchor_shapes, False)
            # Where the overlap is larger than threshold set mask to zero (ignore)
            conf_mask[b, anch_ious > args.ignore_thres] = 0
            # Find the best matching anchor box
            best_n = np.argmax(anch_ious)
            # Get ground truth box
            gt_box = torch.FloatTensor(np.array([gx, gy, gw, gh])).unsqueeze(0)
            # Get the best prediction
            pred_box = pred_boxes[b, best_n, gj, gi].unsqueeze(0)
            # Masks
            mask[b, best_n, gj, gi] = 1
            conf_mask[b, best_n, gj, gi] = 1
            tx[b, best_n, gj, gi] = gx
            ty[b, best_n, gj, gi] = gy
            tw[b, best_n, gj, gi] = gw
            th[b, best_n, gj, gi] = gh
            # One-hot encoding of label
            tcls[b, best_n, gj, gi, int(target[b, t, 0])] = 1
            # Calculate iou between ground truth and best matching prediction
            iou = bbox_iou(gt_box, pred_box, x1y1x2y2=False)
            tconf[b, best_n, gj, gi] = 1

            if iou > 0.5:
                nCorrect += 1
    
    t = torch.stack([tx,ty,tw,th],-1)
    return nGT, nCorrect, mask, conf_mask, t, tconf, tcls

def create_modules(module_defs):
    """
    Constructs module list of layer blocks from module configuration in module_defs
    """
    output_filters = [args.img_channels]
    module_list = nn.ModuleList()
    for i, module_def in enumerate(module_defs):
        modules = nn.Sequential()
        # same_pad = lambda k, s: ((k - 1) * s + 1) // 2 ## Should I use this in padding?
        if module_def['type'] == 'Conv':
            filters = int(module_def['n_fiter'])
            kernel_size = int(module_def['kernel'])
            pad = (kernel_size - 1) // 2 if int(module_def['pad']) else 0
            modules.add_module('conv_%d' % i, nn.Conv2d(in_channels=output_filters[-1],
                                                        out_channels=filters,
                                                        kernel_size=kernel_size,
                                                        stride=int(module_def['stride']),
                                                        padding=pad,
                                                        bias=not module_def['bn']))
            if module_def['bn']:
                modules.add_module('batch_norm_%d' % i, nn.BatchNorm2d(filters))
            if module_def['act'] == 'leaky':
                modules.add_module('leaky_%d' % i, nn.LeakyReLU(0.1))

        elif module_def['type'] == 'upsample':
            upsample = Upsample( scale_factor=int(module_def['stride']),
                                    mode='nearest')
            modules.add_module('upsample_%d' % i, upsample)

        elif module_def['type'] == 'route':
            layers = module_def["layers"]
            filters = sum([output_filters[layer_i] for layer_i in layers])
            modules.add_module('route_%d' % i, EmptyLayer())

        elif module_def['type'] == 'shortcut':
            filters = output_filters[int(module_def['from'])]
            modules.add_module("shortcut_%d" % i, EmptyLayer())

        elif module_def["type"] == "yolo":
            anchor_idxs = module_def["mask"]
            # Extract anchors
            anchors = module_def["anchors"]
            anchors = [(anchors[i], anchors[i+1]) for i in range(0, len(anchors),2)]
            anchors = [anchors[i] for i in anchor_idxs]
            # Define detection layer
            yolo_layer = YOLOLayer(anchors)
            modules.add_module('yolo_%d' % i, yolo_layer)
        # Register module list and number of output filters
        module_list.append(modules)
        output_filters.append(filters)

    return module_list

class EmptyLayer(nn.Module):
    """Placeholder for 'route' and 'shortcut' layers"""
    def __init__(self):
        super(EmptyLayer, self).__init__()

class YOLOLayer(nn.Module):
    """Detection layer"""
    def __init__(self, anchors):
        super(YOLOLayer, self).__init__()
        self.anchors = anchors
        self.num_anchors = args.num_anchors
        self.num_classes = args.num_classes
        self.bbox_attrs = 5 + args.num_classes
        self.ignore_thres = args.ignore_thres
        self.lambda_coord = args.lambda_coord

        self.mse_loss = nn.MSELoss().to(device)
        self.bce_loss = nn.BCELoss().to(device)

    def forward(self, x, targets=None):
        
        '''
        Based on 416x416 coordinate
        '''
        bs = x.size(0)
        n_grids = x.size(2) # n_grids
        grid_size = args.img_size / n_grids 
        # Tensors for cuda support
        FloatTensor = torch.cuda.FloatTensor if x.is_cuda else torch.FloatTensor
        LongTensor = torch.cuda.LongTensor if x.is_cuda else torch.LongTensor

        prediction = x.view(bs, self.num_anchors, self.bbox_attrs, n_grids, n_grids).permute(0, 1, 3, 4, 2).contiguous()
        # (B, num_anchors, g_dim, g_dim, self.bbox_attrs)
        # Get outputs
        x = torch.sigmoid(prediction[..., 0])          # Center x
        y = torch.sigmoid(prediction[..., 1])          # Center y
        w = prediction[..., 2]                         # Width %
        h = prediction[..., 3]                         # Height %
        conf = torch.sigmoid(prediction[..., 4])       # Conf
        pred_cls = torch.sigmoid(prediction[..., 5:])  # Cls pred.

        # Calculate offsets for each grid
        
        grid = torch.arange(n_grids)
        grid_y,grid_x = torch.meshgrid((grid,grid))
        grid_x = grid_x.repeat(bs*args.num_anchors, 1, 1).view(x.shape).float()
        grid_y = grid_y.repeat(bs*args.num_anchors, 1, 1).view(y.shape).float()

        scaled_anchors = self.anchors
        anchor_w = FloatTensor(self.anchors).index_select(1, LongTensor([0]))
        anchor_h = FloatTensor(self.anchors).index_select(1, LongTensor([1]))
        anchor_w = anchor_w.repeat(bs, 1).repeat(1, 1, n_grids*n_grids).view(w.shape)
        anchor_h = anchor_h.repeat(bs, 1).repeat(1, 1, n_grids*n_grids).view(h.shape)

        # Add offset and scale with anchors
        pred_boxes = torch.zeros_like(prediction[..., :4]) 
        pred_boxes[..., 0] = (x.data + grid_x) * grid_size
        pred_boxes[..., 1] = (y.data + grid_y) * grid_size
        pred_boxes[..., 2] = torch.exp(w.data) * anchor_w
        pred_boxes[..., 3] = torch.exp(h.data) * anchor_h
        
        self.output = torch.cat((pred_boxes.view(bs, -1, 4) , conf.view(bs, -1, 1), pred_cls.view(bs, -1, args.num_classes)), -1).detach()
        # Training
        if targets is not None:

            nGT, nCorrect, mask, conf_mask, t, tconf, tcls = build_targets(pred_boxes.cpu().data,
                                                                            targets.cpu().data,
                                                                            self.anchors,
                                                                            args.num_anchors,
                                                                            args.num_classes,
                                                                            n_grids,
                                                                            args.ignore_thres,
                                                                            args.img_size)
            
            
            nProposals = int((conf > 0.25).sum().item())
            recall = float(nCorrect / nGT) if nGT else 1
            mask = mask.float()

            
            t[...,2:] = t[...,2:].sqrt()
            pred_boxes[...,2:] = pred_boxes[...,2:].sqrt()
            
            coord_loss = torch.mean(((t - pred_boxes).pow(2))* mask.unsqueeze(-1).float())
            
            cls_mask = mask.unsqueeze(-1).repeat(1, 1, 1, 1, args.num_classes)
            conf_mask = conf_mask.float()
            loss_conf = self.bce_loss(conf * conf_mask, tconf * conf_mask)
            loss_cls = self.bce_loss(pred_cls * cls_mask, tcls * cls_mask)
            loss = coord_loss + loss_conf + loss_cls

            return loss, coord_loss.item(), loss_conf.item(), loss_cls.item(), recall
        else:
            return None


class Darknet(nn.Module):
    """YOLOv3 object detection model"""
    def __init__(self, module_defs):
        super(Darknet, self).__init__()
        self.module_defs = module_defs
        self.module_list = create_modules(self.module_defs)
        self.img_size = args.img_size
        self.seen = 0
        self.header_info = np.array([0, 0, 0, self.seen, 0])
        self.loss_names = ['coord', 'conf', 'cls', 'recall']
        self.train_hist = defaultdict(list)
        self.apply(self.weight_init)

    def forward(self, x, targets=None):
        is_training = targets is not None
        output = []
        self.pred = []
        self.losses = defaultdict(float)
        layer_outputs = []
        for i, (module_def, module) in enumerate(zip(self.module_defs, self.module_list)):
            if module_def['type'] in ['Conv', 'upsample']:
                x = module(x)
            elif module_def['type'] == 'route':
                layer_i = module_def['layers']
                x = torch.cat([layer_outputs[i] for i in layer_i], 1)
            elif module_def['type'] == 'shortcut':
                layer_i = int(module_def['from'])
                x = layer_outputs[-1] + layer_outputs[layer_i]
            elif module_def['type'] == 'yolo':
                # Train phase: get loss
                if is_training:
                    x, *losses = module[0](x, targets)
                    self.pred.append(module[0].output)
                    for name, loss in zip(self.loss_names, losses):
                        self.losses[name] += loss
                    output.append(x)
                # Test phase: Get detections
                else:
                    x = module(x)
                    self.pred.append(module[0].output)
                    
            layer_outputs.append(x)
            
        self.pred = torch.cat(self.pred,1)

        if is_training:
            for name in self.loss_names:
                self.train_hist[name].append(self.losses[name])
            return sum(output)
    
    def weight_init(self,m):
        if type(m) in [nn.Conv2d, nn.ConvTranspose2d, nn.Linear]:
            #nn.init.xavier_normal_(m.weight,nn.init.calculate_gain('leaky_relu',param=0.02))
            nn.init.kaiming_normal_(m.weight,0.2,nonlinearity='leaky_relu')
    
    def model_save(self,step):
        path = args.model_path + args.model_name +'_Step_' + str(step) + '.pth'
        torch.save({ args.model_name :self.state_dict()}, path)
        print('Model Saved')
        
    def load_step_dict(self, step):
        
        path = args.model_path + args.model_name +'_Step_' + str(step) + '.pth'
        self.load_state_dict(torch.load(path, map_location=lambda storage, loc: storage)[args.model_name])
        print('Model Loaded')
 
    def plot_all_loss(self,step):
        
        fig, ax = plt.subplots(figsize= (20,8))
        for k in self.train_hist.keys():
            plt.plot(self.train_hist[k], label= k)
        plt.ylabel('Loss',fontsize=15)
        plt.xlabel('Number of Steps',fontsize=15)
        plt.title('Loss',fontsize=30,fontweight ="bold")
        plt.legend(loc = 'upper left')
        fig.savefig( args.model_name +"_Loss_"+str(step)+".png")
        
    def num_all_params(self,):
        return sum([param.nelement() for param in self.parameters()])


In [ ]:
coco_d = COCODataset(args.img_path,args.ann_path, transform=to_tensor)
training_loader = DataLoader(coco_d,5,shuffle=True)
dark = Darknet(Model_defs.model_defs).to(device)

epoch = 0
all_steps = 1

In [ ]:
optimizer = optim.Adam(dark.parameters(), lr = args.lr, betas=(args.beta1, args.beta2))
scheduler = optim.lr_scheduler.StepLR(optimizer,10000,0.5)

In [ ]:
# Training 
dark.train()
while epoch < args.n_epoch:
    for i, (path, img, target) in enumerate(training_loader): 
        args.img_p = path # it will be used in both training and detection
        start_t = time.time()
        optimizer.zero_grad()
        img = img.to(device)
        target = target.to(device)
        #scheduler.step()
        loss = dark(img, target)
        loss.backward()
        optimizer.step()
        end_t = time.time()

        
        print('| Step [%d] | lr [%.4f] | Loss: [%.3f] | Time: %.1fs' %\
              ( all_steps, optimizer.param_groups[0]['lr'], loss.item(),
               end_t - start_t))
        
        if all_steps % args.img_show_freq == 0: 
            # Do IoU compress first
            out = non_max_suppression(dark.pred, args.num_classes ,conf_thres=0.5,nms_thres=0.2)
            # Plot the first output
            if out[0] is not None:
                plot_on_o_img([args.img_p[0]], out[0][...,:4].unsqueeze(0), out[0][...,-1].unsqueeze(0),save=True, step= all_steps)
            plt.show()
            dark.plot_all_loss('Training')
            if all_steps % args.model_save_freq == 0: 
                dark.model_save(all_steps)
               
        all_steps += 1
        if all_steps > 0:
            raise StopIteration
    epoch +=1

In [ ]:
############### Detection ###############

detect_img_root = args.img_path
detect_max_objs = 20
n_iters = 0
save_photo = False

detect_dataset = ImageDataset(detect_img_root, to_tensor)
detect_dataloader = DataLoader(detect_dataset, 3, shuffle=False)


def top_confidence(input_list, max_objs = 50, depth = 7):
    from copy import deepcopy
    inputs = deepcopy(input_list)
    for i in range(len(inputs)):
        filled_labels = torch.zeros((max_objs, depth))
        if inputs[i] is not None:
            filled_labels[range(len(inputs[i]))[:max_objs]] = inputs[i][torch.sort(inputs[i][:,4], descending = True)[1][:max_objs].long()]
        inputs[i] = filled_labels
    return torch.stack(inputs)

        
dark.eval()
for i, (d_path, d_img) in enumerate(detect_dataloader):
    
    dark(d_img)
    # do IoU compress first
    outputs = non_max_suppression(dark.pred, args.num_classes ,conf_thres=0.51,nms_thres=0.05)
    outputs = top_confidence(outputs, max_objs = detect_max_objs, depth = 7)
    plot_on_o_img(d_path, outputs[...,:4], outputs[...,-1],save= save_photo)
    
    if n_iters is not None and i >= n_iters:
        raise StopIteration

In [ ]:
# plot_on_o_img(path, target[...,1:], target[...,0])